# [Google Analytics Customer Revenue Prediction](https://www.kaggle.com/c/ga-customer-revenue-prediction)


In [1]:
import numpy as np 
import pandas as pd 
import os
import time
from sklearn.preprocessing import LabelEncoder
import lightgbm
import sklearn.model_selection
import sklearn

In [2]:
TRAIN_FILE_PATH = "../input/train.csv"
TEST_FILE_PATH = "../input/test.csv"
JSON_COLUMNS = ['totals','device','geoNetwork','trafficSource']

### This function opens the input files, flattens the json columns and returns the pandas dataframe

In [3]:
def file_open_flattener(file_path):
    counter = 0
    print ('Flattening the file', file_path)
    data_frame = pd.read_csv(file_path, low_memory=False)
    for json_column in JSON_COLUMNS:
        print('Flattening ' + json_column)
        unpacked = \
            pd.io.json.json_normalize(data_frame[json_column].apply(lambda val: \
                pd.io.json.loads(val)).tolist()).add_prefix(json_column
                + '.')
        data_frame = pd.concat([data_frame, unpacked], axis=1)
        data_frame.drop([json_column], inplace=True, axis=1)
    return data_frame

In [4]:
train_df = file_open_flattener(TRAIN_FILE_PATH)
test_df = file_open_flattener(TEST_FILE_PATH)

Flattening the file ../input/train.csv
Flattening totals
Flattening device
Flattening geoNetwork
Flattening trafficSource
Flattening the file ../input/test.csv
Flattening totals
Flattening device
Flattening geoNetwork
Flattening trafficSource


### Remove columns from train data frame which are not in test data frame except totals.transactionRevenue

In [5]:
for column in train_df.columns:
    if column not in test_df.columns and column != 'totals.transactionRevenue':
        train_df.drop('trafficSource.campaignCode' , axis = 1 , inplace = True)
        print('Dropped ',column)

Dropped  trafficSource.campaignCode


### Remove columns which which only 1 unique element

In [6]:
def remove_non_relevant_columns(dataframe):
    counter = 0
    for column in dataframe.columns:
        if len(dataframe[column].unique()) == 1:
            dataframe.drop(column, axis = 1, inplace = True)
            print ('Column '+ column + ' was dropped')
            counter += 1
    print ('Total Number of Columns dropped :',counter)
    return dataframe

In [7]:
train_df = remove_non_relevant_columns(train_df)
test_df = remove_non_relevant_columns(test_df)

Column socialEngagementType was dropped
Column totals.visits was dropped
Column device.browserSize was dropped
Column device.browserVersion was dropped
Column device.flashVersion was dropped
Column device.language was dropped
Column device.mobileDeviceBranding was dropped
Column device.mobileDeviceInfo was dropped
Column device.mobileDeviceMarketingName was dropped
Column device.mobileDeviceModel was dropped
Column device.mobileInputSelector was dropped
Column device.operatingSystemVersion was dropped
Column device.screenColors was dropped
Column device.screenResolution was dropped
Column geoNetwork.cityId was dropped
Column geoNetwork.latitude was dropped
Column geoNetwork.longitude was dropped
Column geoNetwork.networkLocation was dropped
Column trafficSource.adwordsClickInfo.criteriaParameters was dropped
Total Number of Columns dropped : 19
Column socialEngagementType was dropped
Column totals.visits was dropped
Column device.browserSize was dropped
Column device.browserVersion was

### Date Related Feature Engineering

In [8]:
def date_features_engineering(dataframe):
    dataframe['date'] = dataframe['date'].astype(str)
    dataframe["date"] = dataframe["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    dataframe["date"] = pd.to_datetime(dataframe["date"])
    dataframe["year"] = dataframe['date'].dt.year
    dataframe["month"] = dataframe['date'].dt.month
    dataframe["day"] = dataframe['date'].dt.day
    dataframe["weekday"] = dataframe['date'].dt.weekday
    dataframe['unique_user_count_per_month'] = dataframe.groupby('month')['fullVisitorId'].transform('nunique')
    dataframe['unique_user_count_per_weekday'] = dataframe.groupby('weekday')['fullVisitorId'].transform('nunique')
    return dataframe

In [9]:
train_df = date_features_engineering(train_df)
test_df = date_features_engineering(test_df)

### Preprocess Transaction Revenue
- Changed datatype to float
- Filled missing/invalid values with 0

In [10]:
def preprocess_transactionRevenue_column(dataframe):
    dataframe['totals.transactionRevenue'] = dataframe['totals.transactionRevenue'].astype(float)
    dataframe['totals.transactionRevenue'] = dataframe['totals.transactionRevenue'].fillna(0)
    dataframe['totals.transactionRevenue'] = np.log1p(dataframe['totals.transactionRevenue'])
    return dataframe

In [11]:
train_df = preprocess_transactionRevenue_column(train_df)

### Further data cleaning and type conversion

In [12]:
def type_conversion_preprocessing(dataframe):
    dataframe['totals.pageviews'] = dataframe['visitNumber'].astype(float)
    dataframe['visitNumber'] = dataframe['visitNumber'].astype(float)
    dataframe['totals.hits'] = dataframe['totals.hits'].astype(float)
    dataframe['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    dataframe['totals.pageviews'].fillna(0, inplace=True)
    dataframe['trafficSource.isTrueDirect'].fillna(False, inplace=True)
    return dataframe

In [13]:
train_df = type_conversion_preprocessing(train_df)
test_df = type_conversion_preprocessing(test_df)

### Additional Feature Engineering

In [14]:
def additional_feature_engineering(dataframe):
    dataframe['mean_visits_per_browser'] = np.log1p(dataframe.groupby(['device.browser'])['visitNumber'].transform('mean'))
    dataframe['mean_visits_per_operating_system'] = np.log1p(dataframe.groupby(['device.operatingSystem'])['visitNumber'].transform('mean'))
    dataframe['mean_vists_per_device_category'] = np.log1p(dataframe.groupby(['device.deviceCategory'])['visitNumber'].transform('mean'))
    dataframe['browser_unique_user_count'] = dataframe.groupby('device.browser')['fullVisitorId'].transform('nunique')
    dataframe['os_unique_user_count'] = dataframe.groupby('device.operatingSystem')['fullVisitorId'].transform('nunique')
    dataframe['total_visits_by_user'] = dataframe.groupby(['fullVisitorId'])['visitNumber'].transform('sum')
    dataframe['unique_visits_by_user'] = dataframe.groupby('visitNumber')['fullVisitorId'].transform('nunique')
    dataframe['total_hit_per_user'] = dataframe.groupby(['fullVisitorId'])['totals.hits'].transform('sum')
    dataframe['total_pageviews_per_user'] = dataframe.groupby(['fullVisitorId'])['totals.pageviews'].transform('sum')
    dataframe['mean_hits+per_day'] = dataframe.groupby(['day'])['totals.hits'].transform('mean')
    dataframe['mean_pageviews_per_day'] = dataframe.groupby(['day'])['totals.pageviews'].transform('mean')
    dataframe['mean_visit_per_network_domain']= dataframe.groupby('geoNetwork.networkDomain')['visitNumber'].transform('mean')
    dataframe['mean_pageviews_per_network_domain'] = dataframe.groupby('geoNetwork.networkDomain')['totals.pageviews'].transform('mean')
    dataframe['mean_hits_per_network_domain'] = dataframe.groupby('geoNetwork.networkDomain')['totals.hits'].transform('mean')
    return dataframe

In [15]:
train_df = additional_feature_engineering(train_df)
test_df = additional_feature_engineering(test_df)

### Label Encoding the Categorical Columns

In [16]:
CATEGORICAL_COLUMNS = [
    'channelGrouping', 
    'device.browser', 
    'device.deviceCategory', 
    'device.isMobile', 
    'device.operatingSystem', 
    'geoNetwork.city',
    'geoNetwork.continent',
    'geoNetwork.country',
    'geoNetwork.metro',
    'geoNetwork.networkDomain',
    'geoNetwork.region',
    'geoNetwork.subContinent',
    'totals.bounces',
    'totals.newVisits',
    'trafficSource.adContent',
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.isVideoAd',
    'trafficSource.adwordsClickInfo.page',
    'trafficSource.adwordsClickInfo.slot', 
    'trafficSource.campaign',
    'trafficSource.isTrueDirect',
    'trafficSource.keyword',
    'trafficSource.medium',
    'trafficSource.source',
    'year', 
    'month',
    'day',
    'weekday'
]

In [17]:
def encode_categorical_columns(train_df, test_df):
    for cat_column in CATEGORICAL_COLUMNS:
        print('Processing categorical column:',cat_column)
        label_encoder = LabelEncoder()
        label_encoder.fit(list(train_df[cat_column].values.astype('str')) + list(test_df[cat_column].values.astype('str')))
        train_df[cat_column] = label_encoder.transform(list(train_df[cat_column].values.astype('str')))
        test_df[cat_column] = label_encoder.transform(list(test_df[cat_column].values.astype('str')))
    return train_df, test_df

In [18]:
#train_df, test_df = encode_categorical_columns(train_df, test_df)

### Columns needed for modelling

In [19]:
NEEDED_COLUMNS = [
    'channelGrouping',
    'visitNumber',
    'totals.bounces',
    'totals.hits',
    'totals.newVisits',
    'totals.pageviews',
    'device.browser',
    'device.deviceCategory',
    'device.isMobile',
    'device.operatingSystem',
    'geoNetwork.city',
    'geoNetwork.continent',
    'geoNetwork.country',
    'geoNetwork.metro',
    'geoNetwork.networkDomain',
    'geoNetwork.region',
    'geoNetwork.subContinent',
    'trafficSource.adContent',
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.isVideoAd',
    'trafficSource.adwordsClickInfo.page',
    'trafficSource.adwordsClickInfo.slot',
    'trafficSource.campaign',
    'trafficSource.isTrueDirect',
    'trafficSource.keyword',
    'trafficSource.medium',
    'trafficSource.source',
    'year',
    'month',
    'day',
    'weekday',
    'unique_user_count_per_month',
    'unique_user_count_per_weekday',
    'mean_visits_per_browser',
    'mean_visits_per_operating_system',
    'mean_vists_per_device_category',
    'browser_unique_user_count',
    'os_unique_user_count',
    'total_visits_by_user',
    'unique_visits_by_user',
    'total_hit_per_user',
    'total_pageviews_per_user',
    'mean_hits+per_day',
    'mean_pageviews_per_day',
    'mean_visit_per_network_domain',
    'mean_pageviews_per_network_domain',
    'mean_hits_per_network_domain',
    ]

### LGBM Model Parameters

In [20]:
lgbm_model = lightgbm.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=1000,
    max_bin=255,
    subsample_for_bin=50000,
    objective=None,
    min_split_gain=0,
    min_child_weight=3,
    min_child_samples=10,
    subsample=1,
    subsample_freq=1,
    colsample_bytree=1,
    reg_alpha=0.1,
    reg_lambda=0,
    seed=17,
    silent=False,
    nthread=-1,
    n_jobs=-1
)

NUM_ROUNDS = 150000
VERBOSE_EVAL = 250
STOP_ROUNDS = 500
N_SPLITS = 2

In [21]:
def lgbm_modelling(train_df=None, test_df=None, not_used_cols=None):
    print('K Fold Model Fitting')
    train_df = train_df.sort_values('date')
    X_test = test_df[NEEDED_COLUMNS]
    prediction = np.zeros(test_df.shape[0])
    X = train_df[NEEDED_COLUMNS]
    y = train_df['totals.transactionRevenue']
    for (fold_n, (train_index, test_index)) in \
        enumerate(sklearn.model_selection.KFold(n_splits=N_SPLITS,
                  shuffle=True, random_state=20180917).split(X)):
        print(fold_n)
        (X_train, X_valid) = (X.iloc[train_index], X.iloc[test_index])
        (y_train, y_valid) = (y.iloc[train_index], y.iloc[test_index])
        lgbm_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            eval_metric='rmse',
            verbose=VERBOSE_EVAL,
            early_stopping_rounds=STOP_ROUNDS,
            )
        y_pred = lgbm_model.predict(X_test,
                                    num_iteration=lgbm_model.best_iteration_)
        prediction += y_pred
        prediction /= N_SPLITS
    return prediction

In [22]:
#prediction = lgbm_modelling(train_df, test_df)

In [23]:
def submission(test_df, prediction):
    print('Start to Prepare Submission')
    submission = test_df[['fullVisitorId']].copy()
    submission['PredictedLogRevenue'] = prediction
    submission["PredictedLogRevenue"] = np.expm1(prediction)
    submission = submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
    submission.columns = ["fullVisitorId", "PredictedLogRevenue"]
    submission["PredictedLogRevenue"] = np.log1p(submission["PredictedLogRevenue"])
    submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].fillna(0.0)
    submission.to_csv("submission_final.csv", index=False)
    print('Done with submission')

In [24]:
#submission(test_df, prediction)

In [25]:
train_df['total_visists_per_country'] = train_df.groupby(['geoNetwork.country'])['visitNumber'].transform('sum')

In [26]:
train_df[['total_visists_per_country', 'geoNetwork.country']]

,total_visists_per_country,geoNetwork.country
0,23695.0,Turkey
1,17347.0,Australia
2,19025.0,Spain
3,10665.0,Indonesia
4,65742.0,United Kingdom
5,16302.0,Italy
6,4770.0,Pakistan
7,17347.0,Australia
8,3617.0,Austria
9,18150.0,Netherlands
